In [1]:
import os
from typing import Dict

import torch
from filelock import FileLock
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Normalize, ToTensor
from tqdm import tqdm

import ray.train
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer

In [2]:
def get_dataloaders(batch_size):
    # Transform to normalize the input images
    transform = transforms.Compose([ToTensor(), Normalize((0.5,), (0.5,))])

    with FileLock(os.path.expanduser("~/data.lock")):
        # Download training data from open datasets
        training_data = datasets.FashionMNIST(
            root="~/data",
            train=True,
            download=True,
            transform=transform,
        )

        # Download test data from open datasets
        test_data = datasets.FashionMNIST(
            root="~/data",
            train=False,
            download=True,
            transform=transform,
        )

    # Create data loaders
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader

In [3]:
# Model Definition
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
def train_func_per_worker(config: Dict):
    lr = config["lr"]
    epochs = config["epochs"]
    batch_size = config["batch_size_per_worker"]

    # Get dataloaders inside the worker training function
    train_dataloader, test_dataloader = get_dataloaders(batch_size=batch_size)

    # [1] Prepare Dataloader for distributed training
    # Shard the datasets among workers and move batches to the correct device
    # =======================================================================
    train_dataloader = ray.train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = ray.train.torch.prepare_data_loader(test_dataloader)

    model = NeuralNetwork()

    # [2] Prepare and wrap your model with DistributedDataParallel
    # Move the model to the correct GPU/CPU device
    # ============================================================
    model = ray.train.torch.prepare_model(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Model training loop
    for epoch in range(epochs):
        if ray.train.get_context().get_world_size() > 1:
            # Required for the distributed sampler to shuffle properly across epochs.
            train_dataloader.sampler.set_epoch(epoch)

        model.train()
        for X, y in tqdm(train_dataloader, desc=f"Train Epoch {epoch}"):
            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        test_loss, num_correct, num_total = 0, 0, 0
        with torch.no_grad():
            for X, y in tqdm(test_dataloader, desc=f"Test Epoch {epoch}"):
                pred = model(X)
                loss = loss_fn(pred, y)

                test_loss += loss.item()
                num_total += y.shape[0]
                num_correct += (pred.argmax(1) == y).sum().item()

        test_loss /= len(test_dataloader)
        accuracy = num_correct / num_total

        # [3] Report metrics to Ray Train
        # ===============================
        ray.train.report(metrics={"loss": test_loss, "accuracy": accuracy})


In [5]:
def train_fashion_mnist(num_workers=2, use_gpu=False):
    global_batch_size = 32

    train_config = {
        "lr": 1e-3,
        "epochs": 10,
        "batch_size_per_worker": global_batch_size // num_workers,
    }

    # Configure computation resources
    scaling_config = ScalingConfig(num_workers=num_workers, use_gpu=use_gpu)

    # Initialize a Ray TorchTrainer
    trainer = TorchTrainer(
        train_loop_per_worker=train_func_per_worker,
        train_loop_config=train_config,
        scaling_config=scaling_config,
    )

    # [4] Start distributed training
    # Run `train_func_per_worker` on all workers
    # =============================================
    result = trainer.fit()
    print(f"Training result: {result}")

In [6]:
train_fashion_mnist(num_workers=4, use_gpu=False)

2024-09-29 11:43:02,397	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 
2024-09-29 11:43:02,935	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `<FrameworkTrainer>(...)`.
2024-09-29 11:43:02,936	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-09-29 11:43:03 (running for 00:00:00.13)
Using FIFO scheduling algorithm.
Logical resource usage: 5.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-09-29_11-43-01_585844_53312/artifacts/2024-09-29_11-43-02/TorchTrainer_2024-09-29_11-43-01/driver_artifacts
Number of trials: 1/1 (1 PENDING)




(TorchTrainer pid=53623) Started distributed worker processes: 
(TorchTrainer pid=53623) - (node_id=2ed88b6ebc0fcf280abb7fc3226a29881fbdf419bec129f08b318fd4, ip=127.0.0.1, pid=53630) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=53623) - (node_id=2ed88b6ebc0fcf280abb7fc3226a29881fbdf419bec129f08b318fd4, ip=127.0.0.1, pid=53631) world_rank=1, local_rank=1, node_rank=0
(TorchTrainer pid=53623) - (node_id=2ed88b6ebc0fcf280abb7fc3226a29881fbdf419bec129f08b318fd4, ip=127.0.0.1, pid=53632) world_rank=2, local_rank=2, node_rank=0
(TorchTrainer pid=53623) - (node_id=2ed88b6ebc0fcf280abb7fc3226a29881fbdf419bec129f08b318fd4, ip=127.0.0.1, pid=53633) world_rank=3, local_rank=3, node_rank=0
(RayTrainWorker pid=53630) Setting up process group for: env:// [rank=0, world_size=4]


(RayTrainWorker pid=53630) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(RayTrainWorker pid=53630) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to /Users/atharvakulkarni/data/FashionMNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/26421880 [00:00<?, ?it/s]


...

== Status ==
Current time: 2024-09-29 11:45:29 (running for 00:02:26.89)
Using FIFO scheduling algorithm.
Logical resource usage: 5.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-09-29_11-43-01_585844_53312/artifacts/2024-09-29_11-43-02/TorchTrainer_2024-09-29_11-43-01/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




Train Epoch 9:  62%|██████▏   | 1163/1875 [00:09<00:05, 125.68it/s] [repeated 186x across cluster]


== Status ==
Current time: 2024-09-29 11:45:34 (running for 00:02:31.95)
Using FIFO scheduling algorithm.
Logical resource usage: 5.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-09-29_11-43-01_585844_53312/artifacts/2024-09-29_11-43-02/TorchTrainer_2024-09-29_11-43-01/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




Test Epoch 9: 100%|██████████| 313/313 [00:00<00:00, 1056.42it/s]
2024-09-29 11:45:39,868	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/atharvakulkarni/ray_results/TorchTrainer_2024-09-29_11-43-01' in 0.0033s.
2024-09-29 11:45:39,870	INFO tune.py:1041 -- Total run time: 156.93 seconds (156.90 seconds for the tuning loop).


== Status ==
Current time: 2024-09-29 11:45:39 (running for 00:02:36.90)
Using FIFO scheduling algorithm.
Logical resource usage: 5.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-09-29_11-43-01_585844_53312/artifacts/2024-09-29_11-43-02/TorchTrainer_2024-09-29_11-43-01/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)


Training result: Result(
  metrics={'loss': 0.3730754413829444, 'accuracy': 0.8652},
  path='/Users/atharvakulkarni/ray_results/TorchTrainer_2024-09-29_11-43-01/TorchTrainer_a884f_00000_0_2024-09-29_11-43-02',
  filesystem='local',
  checkpoint=None
)
